# RAG on HATVP

## Install libs

In [ ]:
%%capture
!pip install "unstructured[all-docs]"
!pip install langchain
!pip install openai
!pip install chromadb
!pip install tiktoken

In [ ]:
!pip install langchain-groq

## Download RAW and MASSIVE HATVP declarations

A single XML file containing all declarations.

In [ ]:
!wget https://www.hatvp.fr/livraison/merge/declarations.xml

--2024-04-24 13:54:55--  https://www.hatvp.fr/livraison/merge/declarations.xml
Resolving www.hatvp.fr (www.hatvp.fr)... 185.194.81.47
Connecting to www.hatvp.fr (www.hatvp.fr)|185.194.81.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 124100626 (118M) [text/xml]
Saving to: ‘declarations.xml’

declarations.xml    100%[===================>] 118.35M   576KB/s    in 3m 33s  

2024-04-24 13:58:29 (569 KB/s) - ‘declarations.xml’ saved [124100626/124100626]



## Split large XML into single declaration XML files

In [ ]:
import xml.etree.ElementTree as ET

import os
import xml.etree.ElementTree as ET

def split_xml_documents(file_path):
    # Create a folder named "split_xml" if it doesn't exist
    if not os.path.exists("split_xml"):
        os.makedirs("split_xml")

    # Parse the XML file
    tree = ET.parse(file_path)
    root = tree.getroot()

    # Iterate over each 'declaration' element
    for i, declaration in enumerate(root.findall('declaration'), start=1):
        # Create a new XML tree
        declaration_tree = ET.ElementTree(declaration)
        # Define a new file name
        new_file_name = os.path.join("split_xml", f'declaration_{i:06}.xml')
        # Write the declaration to a new XML file inside the "split_xml" folder
        declaration_tree.write(new_file_name, encoding='utf-8', xml_declaration=True)
        print(f'Created: {new_file_name}')


# Example usage:
# split_xml_documents('path_to_your_large_xml_file.xml')

In [ ]:
xml_large_doc_path = "/content/declarations.xml"
split_xml_documents(xml_large_doc_path)

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
Created: split_xml/declaration_005945.xml
Created: split_xml/declaration_005946.xml
Created: split_xml/declaration_005947.xml
Created: split_xml/declaration_005948.xml
Created: split_xml/declaration_005949.xml
Created: split_xml/declaration_005950.xml
Created: split_xml/declaration_005951.xml
Created: split_xml/declaration_005952.xml
Created: split_xml/declaration_005953.xml
Created: split_xml/declaration_005954.xml
Created: split_xml/declaration_005955.xml
Created: split_xml/declaration_005956.xml
Created: split_xml/declaration_005957.xml
Created: split_xml/declaration_005958.xml
Created: split_xml/declaration_005959.xml
Created: split_xml/declaration_005960.xml
Created: split_xml/declaration_005961.xml
Created: split_xml/declaration_005962.xml
Created: split_xml/declaration_005963.xml
Created: split_xml/declaration_005964.xml
Created: split_xml/declaration_005965.xml
Created: split_xml/declaration_005966.xm

## Test one document (openAI)

In [ ]:
# open test document
with open("/content/split_xml/declaration_000001.xml") as f:
    single_text_declaration = f.read()

In [ ]:
single_text_declaration

"<?xml version='1.0' encoding='utf-8'?>\n<declaration>\n\t\t<dateDepot>11/07/2022 15:40:13</dateDepot>\n\t\t<uuid>4344aaa1-874d-4e6d-9b1a-45f7725b710c</uuid>\n\t\t<origine>ADEL</origine>\n\t\t<complete>true</complete>\n\t\t<attachedFiles>\n\t\t\t<attachedFiles>\n\t\t\t\t<fileName>VUE_PDF_DU_RECEPISSE_DU_DEPOT_XML</fileName>\n\t\t\t\t<serverFileName />\n\t\t\t\t<base64EncodedContent />\n\t\t\t</attachedFiles>\n\t\t</attachedFiles>\n\t\t<declarationVersion>20171221</declarationVersion>\n\t\t<activConsultantDto>\n\t\t\t<neant>true</neant>\n\t\t</activConsultantDto>\n\t\t<activProfCinqDerniereDto>\n\t\t\t<neant>true</neant>\n\t\t</activProfCinqDerniereDto>\n\t\t<activProfConjointDto>\n\t\t\t<items>\n\t\t\t\t<items>\n\t\t\t\t\t<motif>\n\t\t\t\t\t\t<id>CREATION</id>\n\t\t\t\t\t\t<label />\n\t\t\t\t\t</motif>\n\t\t\t\t\t<commentaire />\n\t\t\t\t\t<nomConjoint>\n            [Données non publiées]\n        </nomConjoint>\n\t\t\t\t\t<employeurConjoint>CENTRE HOSPITALIER DU HAUT-BUGEY</employeurC

In [ ]:
# remove line breaks and tabs
single_text_declaration = single_text_declaration.replace('\n', '').replace('\t', '')
single_text_declaration

"<?xml version='1.0' encoding='utf-8'?><declaration><dateDepot>11/07/2022 15:40:13</dateDepot><uuid>4344aaa1-874d-4e6d-9b1a-45f7725b710c</uuid><origine>ADEL</origine><complete>true</complete><attachedFiles><attachedFiles><fileName>VUE_PDF_DU_RECEPISSE_DU_DEPOT_XML</fileName><serverFileName /><base64EncodedContent /></attachedFiles></attachedFiles><declarationVersion>20171221</declarationVersion><activConsultantDto><neant>true</neant></activConsultantDto><activProfCinqDerniereDto><neant>true</neant></activProfCinqDerniereDto><activProfConjointDto><items><items><motif><id>CREATION</id><label /></motif><commentaire /><nomConjoint>            [Données non publiées]        </nomConjoint><employeurConjoint>CENTRE HOSPITALIER DU HAUT-BUGEY</employeurConjoint><activiteProf>Infirmière</activiteProf></items></items><neant>false</neant></activProfConjointDto><fonctionBenevoleDto><neant>true</neant></fonctionBenevoleDto><mandatElectifDto><items><items><motif><id>CREATION</id><label /></motif><comm

In [ ]:
len(single_text_declaration)

11576

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# setup splitter to ignore traditional separators
# chunk_size is optimized for all-MiniLM-L6-v2
# https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    # separators = ["\n\n", "\n", " ", "", "\t"],
    separators = [""],
    keep_separator = False,
    chunk_size=1024,
    chunk_overlap=100,
    length_function=len,
    is_separator_regex=False,
)

In [ ]:
# split test declaration into chunks that are Langchain documents
chunk_docs = text_splitter.create_documents([single_text_declaration])
len(chunk_docs)

13

In [ ]:
import pandas as pd
stats = []
for curr_text in chunk_docs:
  # print(len(curr_text.page_content))
  stats.append(curr_text.page_content)

stats_df = pd.DataFrame(stats, columns = ['raw_text'])
stats_df['text_len'] = stats_df['raw_text'].str.len()
stats_df

,raw_text,text_len
0,<?xml version='1.0' encoding='utf-8'?><declara...,1024
1,ndatElectifDto><items><items><motif><id>CREATI...,1024
2,utNet><montant><montant><annee>2015</annee><mo...,1024
3,022</dateFin></items><items><motif><id>CREATIO...,1024
4,><annee>2017</annee><montant>0</montant></mont...,1024
5,nt></montant><montant><annee>2017</annee><mont...,1024
6,t>07/2017</dateDebut><dateFin /></items><items...,1024
7,aire><nomSociete>ASSOCIATION SAVEURS DE L'AIN<...,1024
8,ciation Les Amis de Damien Abad</activite><rem...,1024
9,/commentaire><nomSociete>CREDIT AGRICOLE SA</n...,1024


In [ ]:
print(stats_df.raw_text.to_list()[0])

<?xml version='1.0' encoding='utf-8'?><declaration><dateDepot>11/07/2022 15:40:13</dateDepot><uuid>4344aaa1-874d-4e6d-9b1a-45f7725b710c</uuid><origine>ADEL</origine><complete>true</complete><attachedFiles><attachedFiles><fileName>VUE_PDF_DU_RECEPISSE_DU_DEPOT_XML</fileName><serverFileName /><base64EncodedContent /></attachedFiles></attachedFiles><declarationVersion>20171221</declarationVersion><activConsultantDto><neant>true</neant></activConsultantDto><activProfCinqDerniereDto><neant>true</neant></activProfCinqDerniereDto><activProfConjointDto><items><items><motif><id>CREATION</id><label /></motif><commentaire /><nomConjoint>            [Données non publiées]        </nomConjoint><employeurConjoint>CENTRE HOSPITALIER DU HAUT-BUGEY</employeurConjoint><activiteProf>Infirmière</activiteProf></items></items><neant>false</neant></activProfConjointDto><fonctionBenevoleDto><neant>true</neant></fonctionBenevoleDto><mandatElectifDto><items><items><motif><id>CREATION</id><label /></motif><comme

In [ ]:
print(stats_df.raw_text.to_list()[1])

ndatElectifDto><items><items><motif><id>CREATION</id><label /></motif><commentaire>REVENUS NETS IMPOSABLES        [Données non publiées]    </commentaire><descriptionMandat>DEPUTE</descriptionMandat><remuneration><brutNet>Net</brutNet><montant><montant><annee>2017</annee><montant>67 047</montant></montant><montant><annee>2018</annee><montant>71 042</montant></montant><montant><annee>2019</annee><montant>71 105</montant></montant><montant><annee>2020</annee><montant>70 773</montant></montant><montant><annee>2021</annee><montant>70 676</montant></montant><montant><annee>2022</annee><montant>27 289</montant></montant></montant></remuneration><dateDebut>01/2017</dateDebut><dateFin /></items><items><motif><id>CREATION</id><label /></motif><commentaire>REVENUS NETS IMPOSABLES        [Données non publiées]    </commentaire><descriptionMandat>PRESIDENT DU DEPARTEMENT 01</descriptionMandat><remuneration><brutNet>Net</brutNet><montant><montant><annee>2015</annee><montant>16865</montant></montant

Summarize test ?

In [ ]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [ ]:
chunk_docs

[Document(page_content="<?xml version='1.0' encoding='utf-8'?><declaration><dateDepot>11/07/2022 15:40:13</dateDepot><uuid>4344aaa1-874d-4e6d-9b1a-45f7725b710c</uuid><origine>ADEL</origine><complete>true</complete><attachedFiles><attachedFiles><fileName>VUE_PDF_DU_RECEPISSE_DU_DEPOT_XML</fileName><serverFileName /><base64EncodedContent /></attachedFiles></attachedFiles><declarationVersion>20171221</declarationVersion><activConsultantDto><neant>true</neant></activConsultantDto><activProfCinqDerniereDto><neant>true</neant></activProfCinqDerniereDto><activProfConjointDto><items><items><motif><id>CREATION</id><label /></motif><commentaire /><nomConjoint>            [Données non publiées]        </nomConjoint><employeurConjoint>CENTRE HOSPITALIER DU HAUT-BUGEY</employeurConjoint><activiteProf>Infirmière</activiteProf></items></items><neant>false</neant></activProfConjointDto><fonctionBenevoleDto><neant>true</neant></fonctionBenevoleDto><mandatElectifDto><items><items><motif><id>CREATION</id

In [ ]:
from langchain.vectorstores.chroma import Chroma
from langchain.embeddings import OpenAIEmbeddings

# use this to delete the previous documents if needed
if False:
  vectorstore.delete_collection()

embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(chunk_docs, embeddings)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [ ]:
query_docs = vectorstore.similarity_search("Quel est le métier du conjoint?", k=10)
query_docs

[Document(page_content="ire /><nom>LOPES MAGNUSON</nom><employeur>Néant</employeur><descriptionActivite>Assistant fonctionnel</descriptionActivite></items><items><motif><id>CREATION</id><label /></motif><commentaire /><nom>FOUGNIES REBECCA</nom><employeur>Néant</employeur><descriptionActivite>Assistante parlementaire locale</descriptionActivite></items></items><neant>false</neant></activCollaborateursDto><observationInteretDto><items><items><motif><id>CREATION</id><label /></motif><commentaire /><contenu>mon équipe de collaborateurs parlementaires sera complétée dans les prochaines semaines</contenu></items></items><neant>false</neant></observationInteretDto><general><typeDeclaration><id>DIA</id><label>Déclaration d'intérêts et d'activités</label></typeDeclaration><mandat><label>Député ou sénateur</label></mandat><qualiteMandat><typeMandat>Député</typeMandat><codCategorieMandat>PAR</codCategorieMandat><nomCategorieMandat>Député ou sénateur</nomCategorieMandat><codTypeMandatFichier>depu

In [ ]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

In [ ]:
from langchain.prompts import ChatPromptTemplate

template = """You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use three sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:
"""
prompt = ChatPromptTemplate.from_template(template)

print(prompt)

input_variables=['context', 'question'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks.\nUse the following pieces of retrieved context to answer the question.\nIf you don't know the answer, just say that you don't know.\nUse three sentences maximum and keep the answer concise.\nQuestion: {question}\nContext: {context}\nAnswer:\n"))]


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

llm = ChatOpenAI(model_name="gpt-4-turbo", temperature=0)

rag_chain = (
    {"context": retriever,  "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
questions_benchmark = [
    "A qui appartient cette declaration?",
    "Quel est le nom du déclarant?",
    "Quel est la rémunération de Damien Abad en qualité de député en 2019 ?",
    "Quelle est l'activité professionnelle du conjoint de Damien Abad?",
    "Quelles sont les entreprises pour lesquelles Mr Abad détient des parts?",
    "Donne moi tous les montants que Mr Abad a perçu."
]

In [ ]:
for curr_q in questions_benchmark:
  print(curr_q)
  print(rag_chain.invoke(curr_q))
  print()

A qui appartient cette declaration?
La déclaration appartient à M. Damien Abad, né le 5 avril 1980.

Quel est le nom du déclarant?
Le nom du déclarant est DAMIEN ABAD.

Quel est la rémunération de Damien Abad en qualité de député en 2019 ?
En 2019, la rémunération nette de Damien Abad en tant que député était de 71 105 euros.

Quelle est l'activité professionnelle du conjoint de Damien Abad?
Le conjoint de Damien Abad travaille comme infirmière au Centre Hospitalier du Haut-Bugey.

Quelles sont les entreprises pour lesquelles Mr Abad détient des parts?
Mr Abad détient des parts dans les entreprises suivantes : ORANGE, CREDIT AGRICOLE SA, AIRBUS, et L'OREAL.

Donne moi tous les montants que Mr Abad a perçu.
Mr. Abad a perçu les montants suivants au cours des années mentionnées :
- En tant que député : 67,047€ en 2017, 71,042€ en 2018, 71,105€ en 2019, 70,773€ en 2020, 70,676€ en 2021, et 27,289€ en 2022.
- En tant que conseiller départemental : 28,007€ en 2017, 24,201€ en 2018, 16,386€ 

We can see that the chunking is efficient (less tokens billed and faster response time) but fails to response to the last question because it would need the entire document to respond.  



## Test the groq version

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

In [ ]:
import os
from google.colab import userdata

os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')

In [ ]:
llm = ChatGroq(temperature=0, model_name="mixtral-8x7b-32768")

rag_chain = (
    {"context": retriever,  "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
for curr_q in questions_benchmark:
  print(curr_q)
  print(rag_chain.invoke(curr_q))
  print()

## No chunking test  

We test various LLMs agains the complete context, no splitting/chunking.

In [ ]:
def get_answer_to_question(question, llm_to_use):

  system = """You are an assistant for question-answering tasks.
  Use the following pieces of retrieved context to answer the question.
  If you don't know the answer, just say that you don't know.
  Use three sentences maximum and keep the answer concise.
  """
  human = "{text}"
  prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

  context = single_text_declaration
  actual_prompt = f"""
  Question: {question}
  Context: {context}
  Answer:
  """

  chain = prompt | llm_to_use | StrOutputParser()
  return chain.invoke({"text": actual_prompt})

In [ ]:
llm_mixtral = ChatGroq(temperature=0, model_name="mixtral-8x7b-32768")

for curr_q in questions_benchmark:
  print(curr_q)
  print(get_answer_to_question(curr_q, llm_mixtral))
  print()

A qui appartient cette declaration?
The declaration belongs to Damien ABAD, a Député (Deputy) from the Ain department (01) in France, based on the context provided.

Quel est le nom du déclarant?
The declarant's name is Damien ABAD.

Quel est la rémunération de Damien Abad en qualité de député en 2019 ?
According to the provided context, Damien Abad's remuneration as a député (member of parliament) in 2019 was 71,105 euros.

Quelle est l'activité professionnelle du conjoint de Damien Abad?
The spouse of Damien Abad works as an "Infirmière" or nurse at the "CENTRE HOSPITALIER DU HAUT-BUGEY."

Quelles sont les entreprises pour lesquelles Mr Abad détient des parts?
The text does not provide information about companies in which Mr. Abad holds shares.

Donne moi tous les montants que Mr Abad a perçu.
Mr. Abad received the following amounts:
- As a Deputy: 67,047 (2017), 71,042 (2018), 71,105 (2019), 70,773 (2020), 70,676 (2021), and 27,289 (2022).
- As President of the Department 01: 16,865

In [ ]:
llm_chatgpt = ChatOpenAI(model_name="gpt-4-turbo", temperature=0)

for curr_q in questions_benchmark:
  print(curr_q)
  print(get_answer_to_question(curr_q, llm_chatgpt))
  print()

A qui appartient cette declaration?
La déclaration appartient à Monsieur Damien Abad.

Quel est le nom du déclarant?
Le nom du déclarant est Damien Abad.

Quel est la rémunération de Damien Abad en qualité de député en 2019 ?
En 2019, la rémunération nette de Damien Abad en tant que député était de 71 105 euros.

Quelle est l'activité professionnelle du conjoint de Damien Abad?
Le conjoint de Damien Abad travaille comme infirmière au Centre Hospitalier du Haut-Bugey.

Quelles sont les entreprises pour lesquelles Mr Abad détient des parts?
Mr. Abad détient des parts dans les entreprises suivantes : ORANGE, CRÉDIT AGRICOLE SA, AIRBUS, et L'ORÉAL.

Donne moi tous les montants que Mr Abad a perçu.
Mr. Abad a perçu les montants suivants au cours des années mentionnées :
- En tant que député : 67 047 € en 2017, 71 042 € en 2018, 71 105 € en 2019, 70 773 € en 2020, 70 676 € en 2021, et 27 289 € en 2022.
- En tant que président du département 01 : 16 865 € en 2015, 23 035 € en 2016, et 20 120 

In [ ]:
dumb_llm_chatgpt = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

for curr_q in questions_benchmark:
  print(curr_q)
  print(get_answer_to_question(curr_q, dumb_llm_chatgpt))
  print()

A qui appartient cette declaration?
Je ne sais pas à qui appartient cette déclaration car le contenu fourni ne mentionne pas explicitement le propriétaire de la déclaration.

Quel est le nom du déclarant?
Le nom du déclarant est Damien Abad.

Quel est la rémunération de Damien Abad en qualité de député en 2019 ?
La rémunération de Damien Abad en qualité de député en 2019 était de 71 105 euros.


Quelle est l'activité professionnelle du conjoint de Damien Abad?
Le conjoint de Damien Abad travaille en tant qu'infirmière au Centre Hospitalier du Haut-Bugey.


Quelles sont les entreprises pour lesquelles Mr Abad détient des parts?
Mr Abad détient des parts dans les entreprises Orange, Crédit Agricole SA, Airbus et L'Oréal.


Donne moi tous les montants que Mr Abad a perçu.
Je ne sais pas.



In [ ]:
llm_llama2_70B = ChatGroq(temperature=0, model_name="llama2-70b-4096")

for curr_q in questions_benchmark:
  print(curr_q)
  print(get_answer_to_question(curr_q, llm_llama2_70B))
  print()

A qui appartient cette declaration?
The person who filled out this form is a public official, specifically the president of the departmental council of Ain. The form lists various positions held by the official, including president of the departmental council, member of the departmental council, and president of the group Les Républicains at the Assemblée Nationale. The form also lists various remunerations and dates for these positions.

The official's name is not explicitly stated in the form, but it can be inferred from the context that it is a person who holds or has held various public offices in the department of Ain.

Quel est le nom du déclarant?
Le déclarant est le président du groupe LR à l'Assemblée nationale.

Il est également président de l'association AINTOURISME, qui développe et promeut la politique touristique du Département de l'Ain.

Il a également été élu président du Département de l'Ain en 2017, fonction qu'il a occupée jusqu'en juillet 2017.

Il est également pré

In [ ]:
llm_llama3_70B = ChatGroq(temperature=0, model_name="llama3-70b-8192")

for curr_q in questions_benchmark:
  print(curr_q)
  print(get_answer_to_question(curr_q, llm_llama3_70B))
  print()

A qui appartient cette declaration?
La déclaration appartient à Damien Abad, député de l'Ain.

Quel est le nom du déclarant?
Le nom du déclarant est DAMIEN ABAD.

Quel est la rémunération de Damien Abad en qualité de député en 2019 ?
La rémunération de Damien Abad en qualité de député en 2019 est de 71 105 euros.

Quelle est l'activité professionnelle du conjoint de Damien Abad?
L'activité professionnelle du conjoint de Damien Abad est infirmière au Centre Hospitalier du Haut-Bugey.

Quelles sont les entreprises pour lesquelles Mr Abad détient des parts?
Mr. Abad holds parts in the following companies: ORANGE, CREDIT AGRICOLE SA, AIRBUS, and L'OREAL.

Donne moi tous les montants que Mr Abad a perçu.
Here are the montants that Mr. Abad received:

* 67,047 (2017)
* 71,042 (2018)
* 71,105 (2019)
* 70,773 (2020)
* 70,676 (2021)
* 27,289 (2022)
* 16,865 (2015)
* 23,035 (2016)
* 20,120 (2017)
* 28,007 (2017)
* 24,201 (2018)
* 16,386 (2019)
* 16,386 (2020)
* 16,384 (2021)
* 6,827 (2022)



In [ ]:
llm_llama3_8B = ChatGroq(temperature=0, model_name="llama3-8b-8192")

for curr_q in questions_benchmark:
  print(curr_q)
  print(get_answer_to_question(curr_q, llm_llama3_8B))
  print()

A qui appartient cette declaration?
La déclaration appartient à Damien Abad, député de l'Ain.

Quel est le nom du déclarant?
The name of the declarant is Damien Abad.

Quel est la rémunération de Damien Abad en qualité de député en 2019 ?
According to the provided context, Damien Abad's remuneration as a deputy in 2019 was 71,105 euros.

Quelle est l'activité professionnelle du conjoint de Damien Abad?
The conjoint's professional activity is Infirmière (nurse).

Quelles sont les entreprises pour lesquelles Mr Abad détient des parts?
According to the provided context, Mr. Abad holds parts in the following companies:

1. Orange
2. Crédit Agricole SA
3. Airbus
4. L'Oréal

Donne moi tous les montants que Mr Abad a perçu.
Mr. Abad has received the following amounts:

* 67,047 euros in 2017
* 71,042 euros in 2018
* 71,105 euros in 2019
* 70,773 euros in 2020
* 70,676 euros in 2021
* 27,289 euros in 2022



## Conclusion

- chunking is an efficient way to respond to very specific questions  
- passing the complete context gives the better answers, but is slow and expensive  
- in both cases (chunking and whole context), we will not be able to handle 10k+ documents.  

If we chunk multiple documents, there is no naive way to be sure that a number belongs to somebody, we will have to add this information to the embedding.  
Same for whole context: there is no way to pass the complete approx 50k tokens context for each question.  


So we have to find something else.  


Possible partial answers:  
- convert XML to JSON before embedding/sending to LLM for a lighter context  
- 2-pass embedding: one for the document and one for the chunk.  
- make a SQL database from the XML files and ask context via SQL commands.